# [실습3] IMDB 영화 후기 데이터셋 벡터화

IMDB 데이터셋을 직접 다운로드하여 벡터화하는 과정을 살펴본다.

준비 과정 1: 데이터셋 다운로드 압축 풀기

압축을 풀면 아래 구조의 디렉토리가 생성된다.

```
aclImdb/
...train/
......pos/
......neg/
...test/
......pos/
......neg/
```

`train`의 `pos`와 `neg` 서브디렉토리에 각각 12,500개의 긍정과 부정 후기가
포함되어 있다.

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 80.2M    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0 80.2M    0  160k    0     0  84926      0  0:16:30  0:00:01  0:16:29 84979
  2 80.2M    2 1824k    0     0   618k      0  0:02:12  0:00:02  0:02:10  618k
  5 80.2M    5 4592k    0     0  1151k      0  0:01:11  0:00:03  0:01:08 1152k
 12 80.2M   12  9.9M    0     0  2055k      0  0:00:39  0:00:04  0:00:35 2056k
 19 80.2M   19 15.5M    0     0  2702k      0  0:00:30  0:00:05  0:00:25 3277k
 27 80.2M   27 21.7M    0     0  3220k      0  0:00:25  0:00:06  0:00:19 4432k
 35 80.2M   35 28.3M    0     0  3659k      0  0:00:22  0:00:07  0:00:15 5455k
 43 80.2M   43 35.1M    0     0  4013k      0  0:00:20  0:00:08  0:00:12 6306k
 52 80.2M   52 41.9M    0     0  4301k      0  0:00

`aclImdb/train/unsup` 서브디렉토리는 필요 없기에 삭제한다.

In [2]:
import platform

if platform.system() == 'Linux':
    !rm -r aclImdb/train/unsup
else:
    import shutil
    unsup_path = './aclImdb/train/unsup'
    shutil.rmtree(unsup_path)

긍정 후기 하나의 내용을 살펴보자.
모델 구성 이전에 훈련 데이터셋을 살펴 보고
모델에 대한 직관을 갖는 과정이 항상 필요하다.

In [ ]:
'''if 'google.colab' in str(get_ipython()):
    !cat aclImdb/train/pos/4077_10.txt
else:
    with open('aclImdb/train/pos/4077_10.txt', 'r') as f:
        text = f.read()
        print(text)'''

In [3]:
file_path = './aclImdb/train/pos/4077_10.txt'

with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()
    print(text)

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy


준비 과정 2: 검증셋 준비

훈련셋의 20%를 검증셋으로 떼어낸다.
이를 위해 `aclImdb/val` 디렉토리를 생성한 후에
긍정과 부정 훈련셋 모두 무작위로 섞은 후 그중 20%를 검증셋 디렉토리로 옮긴다.

In [4]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)            # val 디렉토리 생성
    files = os.listdir(train_dir / category)

    random.Random(1337).shuffle(files)         # 훈련셋 무작위 섞기

    num_val_samples = int(0.2 * len(files))    # 20% 지정 후 검증셋으로 옮기기
    val_files = files[-num_val_samples:]

    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

준비 과정 3: 텐서 데이터셋 준비

`text_dataset_from_directory()` 함수를 이용하여
훈련셋, 검증셋, 테스트셋을 준비한다.
자료형은 모두 `Dataset`이며, 배치 크기는 32를 사용한다.

In [ ]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory( #20000
    "aclImdb/train", batch_size=batch_size
    )

val_ds = keras.utils.text_dataset_from_directory( #5000
    "aclImdb/val", batch_size=batch_size
    )

test_ds = keras.utils.text_dataset_from_directory( #25000
    "aclImdb/test", batch_size=batch_size 
    )

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


각 데이터셋은 배치로 구분되며
입력은 `tf.string` 텐서이고, 타깃은 `int32` 텐서이다.
크기는 모두 32이며 지정된 배치 크기이다.
예를 들어, 첫째 배치의 입력과 타깃 데이터의 정보는 다음과 같다.

In [6]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)

    # 예제: 첫째 배치의 첫째 후기
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])

    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'I just finished watching "El Otro". I have always taken my hat off to Julio Chavez\'s performances, as he is a great actor, but this movie is really depressing and slow. I guess that it would have been even worse if it wasn\'t for Julio. Anyways, this is definitely a film that you will never understand if you are not from Argentina, and even if you are, I would advise you not to rent this movie in order to have a nice time with your girlfriend, boyfriend, family or friends... it is really depressing and incredibly slow, and the plot does not make a lot of sense neither. Probably the director wanted to show the fragility of the human life, but what he does is bore and impress the audience with scenes that shock you a little bit. It gives you something to think about, but not in a good way. Overall, I definitely didn\'t like this movie.', shape=(), dtype=string)

### 11.3.3 시퀀스 활용법

**정수 벡터 데이터셋 준비**

훈련셋의 모든 후기 문장을 정수들의 벡터로 변환한다.
단, 후기 문장이 최대 600개의 단어만 포함하도록 한다.
또한 사용되는 어휘는 빈도 기준 최대 2만 개로 제한한다.

- `max_length = 600`
- `max_tokens = 20000`
- `output_sequence_length=max_length`

In [7]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
# 어휘 색인 생성 대상 훈련셋 후기 텍스트 데이터셋
text_only_train_ds = train_ds.map(lambda x, y: x)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

변환된 첫째 배치의 입력과 타깃 데이터의 정보는 다음과 같다.
`output_sequence_length=600`으로 지정하였기에 모든 문장은 단어를 최대 600개에서
잘린다. 따라서 생성되는 정수들의 벡터는 길이가 모두 600으로 지정된다.
물론 문장이 600개보다 적은 수의 단어를 사용한다면 나머지는 0으로 채워진다.
또한 벡터에 사용된 정수는 2만보다 작은 값이며,
이는 빈도가 가장 높은 2만개의 단어만을 대상(`max_tokens=20000`)으로 했기 때문이다.

In [8]:
for inputs, targets in int_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 600)
inputs.dtype: <dtype: 'int64'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(
[   10   267  1255     8     2  1740    38    10   175  4496    12    11
    18    14     4   326  1861  5480     5     2   836    19     9     7
   417     4   752     3  8921   591    48    24    47   311   342    88
  3565    33  5496  4259     5    17 16181  1526     1     1    37   300
   919  7848    33     2   340   974     1     4   108   285    37    45
    77   206  3721    33   105   390     4     1   234    15    65  6702
   348    37     7   887     6   883    25   201  4838     3  1169    15
    29     5    25  2414   159    90  2608    33   166     6  2485    19
  1761  1908   948   132     2    78     7   607     2  1834    32     8
    32    11     7     4   980     3  3851    20    12    10   421   301
   202     4   565     6   591     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0 

**트랜스포머 구현**

위 그림에서 설명된 트랜스포머 인코더를 층(layer)으로 구현하면 다음과 같다.

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

**트랜스포머 인코더 활용 모델**

훈련 데이터셋이 입력되면 먼저 단어 임베딩을 이용하여
단어들 사이의 연관성을 찾는다.
이후 트랜스포머 인코더로 셀프 어텐션을 적용한다.

In [10]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")

x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

# 길이가 600인 1차원 어레이로 변환
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 256)      │     5,120,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ (None, None, 256)      │       543,776 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,664,033 (21.61 MB)

 Trainable params: 5,664,033 (21.61 MB)

 Non-trainable params: 0 (0.00 B)

훈련 과정은 특별한 게 없다.
테스트셋에 대한 정확도가 87.5% 정도로 바이그램 모델보다 좀 더 낮다.

In [12]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder.keras",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

model = keras.models.load_model(
    "transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder})

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 632s 1s/step - accuracy: 0.5801 - loss: 0.8377 - val_accuracy: 0.8206 - val_loss: 0.3939
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 603s 965ms/step - accuracy: 0.8200 - loss: 0.3916 - val_accuracy: 0.8252 - val_loss: 0.3852
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 630s 1s/step - accuracy: 0.8552 - loss: 0.3391 - val_accuracy: 0.8522 - val_loss: 0.3406
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 642s 1s/step - accuracy: 0.8701 - loss: 0.3111 - val_accuracy: 0.8576 - val_loss: 0.3346
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 643s 1s/step - accuracy: 0.8838 - loss: 0.2844 - val_accuracy: 0.8616 - val_loss: 0.3303
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 665s 1s/step - accuracy: 0.8885 - loss: 0.2668 - val_accuracy: 0.8612 - val_loss: 0.3256
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 594s 950ms/step - accuracy: 0.9014 - loss: 0.2415 - val_accuracy: 0.8608 - val_loss: 0.3369
Epoch 8/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 635s 1s/step - accuracy: 0.9114 - loss: 0.2267 - va

c:\Users\302-26\anaconda3\Lib\site-packages\keras\src\layers\layer.py:396: UserWarning: `build()` was called on layer 'transformer_encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


782/782 ━━━━━━━━━━━━━━━━━━━━ 241s 307ms/step - accuracy: 0.8676 - loss: 0.3109
Test acc: 0.869


**단어 위치 인코딩**

다음 `PositionalEmbedding` 층 클래스는 두 개의 임베딩 클래스를 사용한다.
하나는 보통의 단어 임베딩이며,
다른 하나는 단어의 위치 정보를 임베딩한다.
각 임베딩의 출력값을 합친 값을 트랜스포머에게 전달하는 역할을 수행한다.

In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size,
            output_dim=embed_dim,
            mask_zero=True  # ✅ 마스크 자동 생성
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length,
            output_dim=embed_dim
        )

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        positions = self.position_embeddings(positions)
        embedded_tokens = self.token_embeddings(inputs)
        return embedded_tokens + positions

    def compute_mask(self, inputs, mask=None):
        # ✅ 직접 마스크 처리하지 않고 Embedding의 마스크를 그대로 전달
        return self.token_embeddings.compute_mask(inputs)


**단어위치인식 트랜스포머 아키텍처**

아래 코드는 `PositionalEmbedding` 층을 활용하여 트랜스포머 인코더가
단어위치를 활용할 수 있도록 한다.

In [19]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")

# PositionalEmbedding 레이어는 mask_zero=True인 Embedding 포함해야 함
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)

# Transformer 인코더 레이어
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

c:\Users\302-26\anaconda3\Lib\site-packages\keras\src\layers\layer.py:940: UserWarning: Layer 'transformer_encoder_1' (of type TransformerEncoder) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [20]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │  5,273,600 │ input_layer_6[0]… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None)      │          0 │ input_layer_6[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, None, 256) │    543,776 │ positional_embed… │
│ (TransformerEncode… │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 256)       │          0 │ transformer_enco… │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 256)       │          0 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 1)         │        257 │ dropout_4[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,817,633 (22.19 MB)

 Trainable params: 5,817,633 (22.19 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/20


c:\Users\302-26\anaconda3\Lib\site-packages\keras\src\layers\layer.py:940: UserWarning: Layer 'transformer_encoder_1' (of type TransformerEncoder) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


625/625 ━━━━━━━━━━━━━━━━━━━━ 665s 1s/step - accuracy: 0.6385 - loss: 0.7597 - val_accuracy: 0.8126 - val_loss: 0.4129
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 626s 1s/step - accuracy: 0.8522 - loss: 0.3434 - val_accuracy: 0.8342 - val_loss: 0.4186
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 627s 1s/step - accuracy: 0.8855 - loss: 0.2748 - val_accuracy: 0.8730 - val_loss: 0.3097
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 630s 1s/step - accuracy: 0.9084 - loss: 0.2311 - val_accuracy: 0.8682 - val_loss: 0.3542
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 619s 991ms/step - accuracy: 0.9241 - loss: 0.1981 - val_accuracy: 0.8518 - val_loss: 0.4928
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 687s 1s/step - accuracy: 0.9384 - loss: 0.1601 - val_accuracy: 0.8802 - val_loss: 0.3840
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 686s 1s/step - accuracy: 0.9503 - loss: 0.1357 - val_accuracy: 0.8820 - val_loss: 0.4192
Epoch 8/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 684s 1s/step - accuracy: 0.9628 - loss: 0.1016 - val_accuracy: 0.

KeyboardInterrupt: 